## Continue ensembles from v1 on full dataset format commonly used on Kaggle

### Integrated mlflow lets this notebook rinse-repeat

In [1]:
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
from collections import Counter
import mlflow

# local imports
from prepare import *
from evaluate import *

### Set new experiment to keep track in mlflow

In [2]:
mlflow.set_experiment('366_features')

INFO: '366_features' does not exist. Creating a new experiment


### Read in initial datasets if needed

In [3]:
#raw_train, raw_train_labels, raw_test, specs, sample = read_raw_csvs()
#raw_train_labels = pd.read_csv('data/train_labels.csv')

### Load large train/test features from Josh's work

In [4]:
reduced_train = pd.read_csv('reduce_train.csv')
reduced_test = pd.read_csv('reduce_test.csv')
reduced_train.shape, reduced_test.shape

((17690, 890), (1000, 890))

### Start throwing model mud at the wall

In [5]:
from catboost import CatBoostClassifier
from sklearn.ensemble import (
    RandomForestClassifier,
    AdaBoostClassifier,
    ExtraTreesClassifier,
    BaggingClassifier,
    GradientBoostingClassifier,
    VotingClassifier,
    StackingClassifier,
)
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, SGDClassifier, RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
import warnings

warnings.filterwarnings(
    "ignore"
)  # Ridge classifier throws some warnings about ill-conditioned matrix

C:\Users\604572\Continuum\anaconda3\envs\dsb2019\lib\imp.py:343: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  return _load(spec)


### Baseline accuracy of 50% (or 25% if balanced)

In [6]:
reduced_train.accuracy_group.value_counts(normalize=True)

3    0.500000
0    0.239062
1    0.136292
2    0.124647
Name: accuracy_group, dtype: float64

# Initialize Models and start testing accuracy

In [7]:
rf = RandomForestClassifier()
rc = RidgeClassifier()
ac = AdaBoostClassifier()
et = ExtraTreesClassifier()
bc = BaggingClassifier()
gbc = GradientBoostingClassifier()

clf = CatBoostClassifier(
    loss_function='MultiClass',
    task_type="CPU",
    learning_rate=0.05,
    iterations=2000,
    od_type="Iter",
    early_stopping_rounds=500,
    random_seed=42
    )

# tried with minimal results
#nb = GaussianNB()
#lr = LogisticRegression()               ## if they are commented out, they weren't performing well (or operator error...)
#sgd = SGDClassifier()

In [8]:
knn = KNeighborsClassifier()
svc = SVC(probability=True, verbose=1)

## Evaluate model performance

In [9]:
# Catboost - accuracy 56%a
clf = quick_eval(reduced_train, clf, cv=True)#, cv=True)

0:	learn: 1.3516310	total: 122ms	remaining: 4m 4s
1:	learn: 1.3213455	total: 179ms	remaining: 2m 58s
2:	learn: 1.2949604	total: 253ms	remaining: 2m 48s
3:	learn: 1.2712020	total: 318ms	remaining: 2m 38s
4:	learn: 1.2499525	total: 388ms	remaining: 2m 34s
5:	learn: 1.2310873	total: 456ms	remaining: 2m 31s
6:	learn: 1.2147990	total: 515ms	remaining: 2m 26s
7:	learn: 1.2002074	total: 589ms	remaining: 2m 26s
8:	learn: 1.1874148	total: 649ms	remaining: 2m 23s
9:	learn: 1.1749735	total: 711ms	remaining: 2m 21s
10:	learn: 1.1623493	total: 774ms	remaining: 2m 19s
11:	learn: 1.1522881	total: 832ms	remaining: 2m 17s
12:	learn: 1.1433888	total: 885ms	remaining: 2m 15s
13:	learn: 1.1343182	total: 949ms	remaining: 2m 14s
14:	learn: 1.1270507	total: 1.01s	remaining: 2m 13s
15:	learn: 1.1202035	total: 1.08s	remaining: 2m 14s
16:	learn: 1.1132417	total: 1.15s	remaining: 2m 14s
17:	learn: 1.1066038	total: 1.22s	remaining: 2m 14s
18:	learn: 1.0997954	total: 1.29s	remaining: 2m 14s
19:	learn: 1.0938704	to

160:	learn: 0.9638092	total: 9.5s	remaining: 1m 48s
161:	learn: 0.9635450	total: 9.58s	remaining: 1m 48s
162:	learn: 0.9631057	total: 9.67s	remaining: 1m 48s
163:	learn: 0.9627654	total: 9.77s	remaining: 1m 49s
164:	learn: 0.9623871	total: 9.84s	remaining: 1m 49s
165:	learn: 0.9618382	total: 9.92s	remaining: 1m 49s
166:	learn: 0.9616509	total: 10s	remaining: 1m 49s
167:	learn: 0.9611485	total: 10.1s	remaining: 1m 49s
168:	learn: 0.9605543	total: 10.1s	remaining: 1m 49s
169:	learn: 0.9603213	total: 10.2s	remaining: 1m 49s
170:	learn: 0.9600726	total: 10.3s	remaining: 1m 49s
171:	learn: 0.9597217	total: 10.3s	remaining: 1m 49s
172:	learn: 0.9593527	total: 10.4s	remaining: 1m 50s
173:	learn: 0.9591372	total: 10.5s	remaining: 1m 50s
174:	learn: 0.9589031	total: 10.6s	remaining: 1m 50s
175:	learn: 0.9586389	total: 10.6s	remaining: 1m 50s
176:	learn: 0.9583280	total: 10.7s	remaining: 1m 50s
177:	learn: 0.9579994	total: 10.8s	remaining: 1m 50s
178:	learn: 0.9575874	total: 10.8s	remaining: 1m 

317:	learn: 0.9039739	total: 19.1s	remaining: 1m 40s
318:	learn: 0.9037833	total: 19.1s	remaining: 1m 40s
319:	learn: 0.9033491	total: 19.2s	remaining: 1m 40s
320:	learn: 0.9030467	total: 19.2s	remaining: 1m 40s
321:	learn: 0.9029136	total: 19.3s	remaining: 1m 40s
322:	learn: 0.9027594	total: 19.3s	remaining: 1m 40s
323:	learn: 0.9026014	total: 19.4s	remaining: 1m 40s
324:	learn: 0.9022173	total: 19.4s	remaining: 1m 40s
325:	learn: 0.9020025	total: 19.5s	remaining: 1m 40s
326:	learn: 0.9017296	total: 19.5s	remaining: 1m 39s
327:	learn: 0.9014692	total: 19.6s	remaining: 1m 39s
328:	learn: 0.9011604	total: 19.6s	remaining: 1m 39s
329:	learn: 0.9009979	total: 19.7s	remaining: 1m 39s
330:	learn: 0.9005763	total: 19.8s	remaining: 1m 39s
331:	learn: 0.9003217	total: 19.8s	remaining: 1m 39s
332:	learn: 0.9000278	total: 19.9s	remaining: 1m 39s
333:	learn: 0.8995792	total: 19.9s	remaining: 1m 39s
334:	learn: 0.8993321	total: 20s	remaining: 1m 39s
335:	learn: 0.8991385	total: 20s	remaining: 1m 3

474:	learn: 0.8651569	total: 27.4s	remaining: 1m 28s
475:	learn: 0.8649611	total: 27.5s	remaining: 1m 27s
476:	learn: 0.8648219	total: 27.5s	remaining: 1m 27s
477:	learn: 0.8646545	total: 27.6s	remaining: 1m 27s
478:	learn: 0.8644074	total: 27.6s	remaining: 1m 27s
479:	learn: 0.8640724	total: 27.7s	remaining: 1m 27s
480:	learn: 0.8639143	total: 27.7s	remaining: 1m 27s
481:	learn: 0.8634620	total: 27.8s	remaining: 1m 27s
482:	learn: 0.8633673	total: 27.8s	remaining: 1m 27s
483:	learn: 0.8631328	total: 27.9s	remaining: 1m 27s
484:	learn: 0.8628460	total: 27.9s	remaining: 1m 27s
485:	learn: 0.8627122	total: 28s	remaining: 1m 27s
486:	learn: 0.8625031	total: 28s	remaining: 1m 27s
487:	learn: 0.8624039	total: 28.1s	remaining: 1m 27s
488:	learn: 0.8621108	total: 28.1s	remaining: 1m 26s
489:	learn: 0.8619201	total: 28.2s	remaining: 1m 26s
490:	learn: 0.8617344	total: 28.2s	remaining: 1m 26s
491:	learn: 0.8615255	total: 28.3s	remaining: 1m 26s
492:	learn: 0.8613137	total: 28.3s	remaining: 1m 2

631:	learn: 0.8333561	total: 35.9s	remaining: 1m 17s
632:	learn: 0.8330056	total: 36s	remaining: 1m 17s
633:	learn: 0.8327263	total: 36.1s	remaining: 1m 17s
634:	learn: 0.8326531	total: 36.1s	remaining: 1m 17s
635:	learn: 0.8325452	total: 36.1s	remaining: 1m 17s
636:	learn: 0.8324534	total: 36.2s	remaining: 1m 17s
637:	learn: 0.8322471	total: 36.2s	remaining: 1m 17s
638:	learn: 0.8319494	total: 36.3s	remaining: 1m 17s
639:	learn: 0.8317999	total: 36.3s	remaining: 1m 17s
640:	learn: 0.8316969	total: 36.4s	remaining: 1m 17s
641:	learn: 0.8315383	total: 36.4s	remaining: 1m 17s
642:	learn: 0.8314995	total: 36.5s	remaining: 1m 17s
643:	learn: 0.8313863	total: 36.5s	remaining: 1m 16s
644:	learn: 0.8311819	total: 36.6s	remaining: 1m 16s
645:	learn: 0.8310441	total: 36.7s	remaining: 1m 16s
646:	learn: 0.8308665	total: 36.7s	remaining: 1m 16s
647:	learn: 0.8308042	total: 36.8s	remaining: 1m 16s
648:	learn: 0.8306963	total: 36.8s	remaining: 1m 16s
649:	learn: 0.8304961	total: 36.9s	remaining: 1m

...

1313:	learn: 0.7390653	total: 1m 12s	remaining: 37.7s
1314:	learn: 0.7388903	total: 1m 12s	remaining: 37.6s
1315:	learn: 0.7387343	total: 1m 12s	remaining: 37.6s
1316:	learn: 0.7386263	total: 1m 12s	remaining: 37.5s
1317:	learn: 0.7384599	total: 1m 12s	remaining: 37.5s
1318:	learn: 0.7384362	total: 1m 12s	remaining: 37.4s
1319:	learn: 0.7384117	total: 1m 12s	remaining: 37.4s
1320:	learn: 0.7382724	total: 1m 12s	remaining: 37.3s
1321:	learn: 0.7382011	total: 1m 12s	remaining: 37.2s
1322:	learn: 0.7379501	total: 1m 12s	remaining: 37.2s
1323:	learn: 0.7378446	total: 1m 12s	remaining: 37.1s
1324:	learn: 0.7377121	total: 1m 12s	remaining: 37.1s
1325:	learn: 0.7376364	total: 1m 12s	remaining: 37s
1326:	learn: 0.7375975	total: 1m 12s	remaining: 37s
1327:	learn: 0.7374646	total: 1m 12s	remaining: 36.9s
1328:	learn: 0.7373523	total: 1m 13s	remaining: 36.9s
1329:	learn: 0.7371801	total: 1m 13s	remaining: 36.8s
1330:	learn: 0.7369703	total: 1m 13s	remaining: 36.8s
1331:	learn: 0.7368767	total: 1m

1466:	learn: 0.7195899	total: 1m 20s	remaining: 29.2s
1467:	learn: 0.7195253	total: 1m 20s	remaining: 29.2s
1468:	learn: 0.7194348	total: 1m 20s	remaining: 29.1s
1469:	learn: 0.7192892	total: 1m 20s	remaining: 29.1s
1470:	learn: 0.7191774	total: 1m 20s	remaining: 29s
1471:	learn: 0.7191121	total: 1m 20s	remaining: 29s
1472:	learn: 0.7190407	total: 1m 20s	remaining: 28.9s
1473:	learn: 0.7189173	total: 1m 20s	remaining: 28.9s
1474:	learn: 0.7188415	total: 1m 20s	remaining: 28.8s
1475:	learn: 0.7186774	total: 1m 20s	remaining: 28.7s
1476:	learn: 0.7186227	total: 1m 21s	remaining: 28.7s
1477:	learn: 0.7184918	total: 1m 21s	remaining: 28.6s
1478:	learn: 0.7184568	total: 1m 21s	remaining: 28.6s
1479:	learn: 0.7183034	total: 1m 21s	remaining: 28.5s
1480:	learn: 0.7181309	total: 1m 21s	remaining: 28.5s
1481:	learn: 0.7179734	total: 1m 21s	remaining: 28.4s
1482:	learn: 0.7178891	total: 1m 21s	remaining: 28.4s
1483:	learn: 0.7177791	total: 1m 21s	remaining: 28.3s
1484:	learn: 0.7175629	total: 1m

1622:	learn: 0.7018200	total: 1m 28s	remaining: 20.7s
1623:	learn: 0.7017380	total: 1m 29s	remaining: 20.6s
1624:	learn: 0.7015710	total: 1m 29s	remaining: 20.6s
1625:	learn: 0.7015121	total: 1m 29s	remaining: 20.5s
1626:	learn: 0.7014436	total: 1m 29s	remaining: 20.4s
1627:	learn: 0.7012646	total: 1m 29s	remaining: 20.4s
1628:	learn: 0.7010743	total: 1m 29s	remaining: 20.3s
1629:	learn: 0.7008668	total: 1m 29s	remaining: 20.3s
1630:	learn: 0.7008012	total: 1m 29s	remaining: 20.2s
1631:	learn: 0.7006884	total: 1m 29s	remaining: 20.2s
1632:	learn: 0.7005875	total: 1m 29s	remaining: 20.1s
1633:	learn: 0.7005351	total: 1m 29s	remaining: 20.1s
1634:	learn: 0.7004428	total: 1m 29s	remaining: 20s
1635:	learn: 0.7002291	total: 1m 29s	remaining: 20s
1636:	learn: 0.7001447	total: 1m 29s	remaining: 19.9s
1637:	learn: 0.7000411	total: 1m 29s	remaining: 19.8s
1638:	learn: 0.6999535	total: 1m 29s	remaining: 19.8s
1639:	learn: 0.6998654	total: 1m 29s	remaining: 19.7s
1640:	learn: 0.6997484	total: 1m

1775:	learn: 0.6855337	total: 1m 36s	remaining: 12.2s
1776:	learn: 0.6854902	total: 1m 36s	remaining: 12.2s
1777:	learn: 0.6854454	total: 1m 36s	remaining: 12.1s
1778:	learn: 0.6853652	total: 1m 36s	remaining: 12s
1779:	learn: 0.6852707	total: 1m 36s	remaining: 12s
1780:	learn: 0.6851464	total: 1m 37s	remaining: 11.9s
1781:	learn: 0.6850049	total: 1m 37s	remaining: 11.9s
1782:	learn: 0.6849001	total: 1m 37s	remaining: 11.8s
1783:	learn: 0.6847923	total: 1m 37s	remaining: 11.8s
1784:	learn: 0.6847224	total: 1m 37s	remaining: 11.7s
1785:	learn: 0.6846515	total: 1m 37s	remaining: 11.7s
1786:	learn: 0.6845151	total: 1m 37s	remaining: 11.6s
1787:	learn: 0.6844182	total: 1m 37s	remaining: 11.5s
1788:	learn: 0.6843808	total: 1m 37s	remaining: 11.5s
1789:	learn: 0.6843104	total: 1m 37s	remaining: 11.4s
1790:	learn: 0.6841439	total: 1m 37s	remaining: 11.4s
1791:	learn: 0.6839778	total: 1m 37s	remaining: 11.3s
1792:	learn: 0.6838553	total: 1m 37s	remaining: 11.3s
1793:	learn: 0.6837537	total: 1m

1928:	learn: 0.6678527	total: 1m 46s	remaining: 3.9s
1929:	learn: 0.6676168	total: 1m 46s	remaining: 3.85s
1930:	learn: 0.6675838	total: 1m 46s	remaining: 3.79s
1931:	learn: 0.6673975	total: 1m 46s	remaining: 3.74s
1932:	learn: 0.6672762	total: 1m 46s	remaining: 3.68s
1933:	learn: 0.6672091	total: 1m 46s	remaining: 3.63s
1934:	learn: 0.6671330	total: 1m 46s	remaining: 3.57s
1935:	learn: 0.6670628	total: 1m 46s	remaining: 3.52s
1936:	learn: 0.6670001	total: 1m 46s	remaining: 3.46s
1937:	learn: 0.6669383	total: 1m 46s	remaining: 3.41s
1938:	learn: 0.6668130	total: 1m 46s	remaining: 3.35s
1939:	learn: 0.6667091	total: 1m 46s	remaining: 3.3s
1940:	learn: 0.6666526	total: 1m 46s	remaining: 3.25s
1941:	learn: 0.6665491	total: 1m 46s	remaining: 3.19s
1942:	learn: 0.6664806	total: 1m 46s	remaining: 3.14s
1943:	learn: 0.6663963	total: 1m 46s	remaining: 3.08s
1944:	learn: 0.6662738	total: 1m 47s	remaining: 3.03s
1945:	learn: 0.6662037	total: 1m 47s	remaining: 2.97s
1946:	learn: 0.6661371	total: 

In [10]:
rf = quick_eval(reduced_train, rf, cv=True)  #.563cv acc
rc = quick_eval(reduced_train, rc, cv=True)  #.528cv acc
ac = quick_eval(reduced_train, ac, cv=True)  #.529cv acc
et = quick_eval(reduced_train, et, cv=True)  #.5399cv acc
bc = quick_eval(reduced_train, bc, cv=True)  #.511cv acc
gbc = quick_eval(reduced_train, gbc, cv=True)  #.435 cv acc



The CV qwk score of RandomForestClassifier is 0.4448587925343516
The CV qwk score of RidgeClassifier is 0.28709891617642075
The CV qwk score of AdaBoostClassifier is 0.4845023860612449
The CV qwk score of ExtraTreesClassifier is 0.380016730391244
The CV qwk score of BaggingClassifier is 0.4734209359489448
The CV qwk score of GradientBoostingClassifier is 0.5120674681117273


In [11]:
#quick_eval(reduced_train, mlp, cv=True)   # 0.295
#quick_eval(reduced_train, gpc) #   .498
#quick_eval(reduced_train, rbf, cv=True)   poor
#quick_eval(reduced_train, dt, cv=True)     #.411

In [12]:

bc_rf_5 = BaggingClassifier(
    base_estimator=RandomForestClassifier(max_depth=5),
    n_estimators=20)#The accuracy of BaggingClassifier is 0.5740531373657434
                    #The QWK of BaggingClassifier is 0.40494994168142406

bc_gbc = BaggingClassifier(
    base_estimator=GradientBoostingClassifier(),
    n_estimators=20)
# The accuracy of BaggingClassifier is 0.6237987563595252
# The QWK of BaggingClassifier is 0.5252575839341429

bc_abc = BaggingClassifier(
    base_estimator=AdaBoostClassifier(),
    n_estimators=20)
# The accuracy of BaggingClassifier is 0.610231769361221
# The QWK of BaggingClassifier is 0.4936903908658856

bc_rf_5 = quick_eval(reduced_train, bc_rf_5)
# quick_eval(reduced_train, bc_rf_10)    # 0.549 with 20 estimators max depth of 10  (.377 with balanced)    0.551 with 200 
# quick_eval(reduced_train, bc_rf_50) 
bc_gbc = quick_eval(reduced_train, bc_gbc)   # 0.576 with 20 estimators (.389 with balanced)                     0.578 with 200
bc_abc = quick_eval(reduced_train, bc_abc)   # 0.563 with 20 estimators  (.374 with balanced)                    0.561 with 200

The accuracy of BaggingClassifier is 0.5491803278688525
The QWK of BaggingClassifier is 0.224878455927486
The accuracy of BaggingClassifier is 0.6192764273600905
The QWK of BaggingClassifier is 0.5207483657285402
The accuracy of BaggingClassifier is 0.6088185415488977
The QWK of BaggingClassifier is 0.4979560762316827


## Model Ensembling

In [13]:
estimators = [
    ('Adaboost',ac),
    ('rf', rf), 
    ('gbc', gbc),
    ('et', et),

    ('rc', rc),

    ('bc_rf_5', bc_rf_5),

    ('bc_gbc', bc_gbc),
    ('bc_abc', bc_abc),
]

stacking_clf = StackingClassifier(estimators=estimators, 
                final_estimator=RandomForestClassifier(), n_jobs=-1)        # created, load from pickle
quick_eval(reduced_train, stacking_clf)

The accuracy of StackingClassifier is 0.623160340821069
The QWK of StackingClassifier is 0.5280815828800287


StackingClassifier(cv=None,
                   estimators=[('Adaboost',
                                AdaBoostClassifier(algorithm='SAMME.R',
                                                   base_estimator=None,
                                                   learning_rate=1.0,
                                                   n_estimators=50,
                                                   random_state=None)),
                               ('rf',
                                RandomForestClassifier(bootstrap=True,
                                                       ccp_alpha=0.0,
                                                       class_weight=None,
                                                       criterion='gini',
                                                       max_depth=None,
                                                       max_features='auto',
                                                       max_leaf_nodes=None,
                         

In [14]:
vc = VotingClassifier(estimators=estimators,
     n_jobs=-1,
     voting='hard')

quick_eval(reduced_train, vc)#, cv=True)     #initial w/o catboost .564

The accuracy of VotingClassifier is 0.6219984508133231
The QWK of VotingClassifier is 0.5101809311828125


VotingClassifier(estimators=[('Adaboost',
                              AdaBoostClassifier(algorithm='SAMME.R',
                                                 base_estimator=None,
                                                 learning_rate=1.0,
                                                 n_estimators=50,
                                                 random_state=None)),
                             ('rf',
                              RandomForestClassifier(bootstrap=True,
                                                     ccp_alpha=0.0,
                                                     class_weight=None,
                                                     criterion='gini',
                                                     max_depth=None,
                                                     max_features='auto',
                                                     max_leaf_nodes=None,
                                                     max_samples=None,
          

In [15]:
vc = VotingClassifier(estimators=estimators,
     n_jobs=-1,
     voting='soft')

quick_eval(reduced_train, vc)#, cv=True)     #initial w/o catboost .564

AttributeError: 'RidgeClassifier' object has no attribute 'predict_proba'